In [7]:
import re
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException  # Import the exception
import time
import base64
import os

In [4]:
headers = {'User-Agent': 'Mozilla/5.0'}

def fetch_page(url):
    response = requests.get(url, headers=headers)
    return BeautifulSoup(response.text, 'html.parser')

def find_player_links(team_url):
    soup = fetch_page(team_url)
    player_links = set()  # Use a set to automatically remove duplicates
    for a_tag in soup.find_all('a', href=True):
        if re.match(r'/player/.+/\d+', a_tag['href']):
            full_url = 'https://www.sofascore.com' + a_tag['href']
            player_links.add(full_url)
    return list(player_links)  # Convert back to list if needed

# List of team URLs
team_urls = [
    'https://www.sofascore.com/team/football/arsenal/42',
    'https://www.sofascore.com/team/football/liverpool/44',
    'https://www.sofascore.com/team/football/manchester-city/17',
    'https://www.sofascore.com/team/football/aston-villa/40',
    'https://www.sofascore.com/team/football/tottenham-hotspur/33',
    'https://www.sofascore.com/team/football/manchester-united/35',
    'https://www.sofascore.com/team/football/west-ham-united/37',
    'https://www.sofascore.com/team/football/brighton-and-hove-albion/30',
    'https://www.sofascore.com/team/football/wolverhampton/3',
    'https://www.sofascore.com/team/football/newcastle-united/39',
    'https://www.sofascore.com/team/football/chelsea/38',
    'https://www.sofascore.com/team/football/fulham/43',
    'https://www.sofascore.com/team/football/bournemouth/60',
    'https://www.sofascore.com/team/football/crystal-palace/7',
    'https://www.sofascore.com/team/football/brentford/50',
    'https://www.sofascore.com/team/football/everton/48',
    'https://www.sofascore.com/team/football/luton-town/72',
    'https://www.sofascore.com/team/football/nottingham-forest/14',
    'https://www.sofascore.com/team/football/burnley/6',
    'https://www.sofascore.com/team/football/sheffield-united/15',
    'https://www.sofascore.com/team/football/real-madrid/2829',
    'https://www.sofascore.com/team/football/barcelona/2817',
    'https://www.sofascore.com/team/football/girona-fc/24264',
    'https://www.sofascore.com/team/football/athletic-club/2825',
    'https://www.sofascore.com/team/football/atletico-madrid/2836',
    'https://www.sofascore.com/team/football/real-sociedad/2824',
    'https://www.sofascore.com/team/football/real-betis/2816',
    'https://www.sofascore.com/team/football/valencia/2828',
    'https://www.sofascore.com/team/football/villarreal/2819',
    'https://www.sofascore.com/team/football/getafe/2859',
    'https://www.sofascore.com/team/football/las-palmas/6577',
    'https://www.sofascore.com/team/football/osasuna/2820',
    'https://www.sofascore.com/team/football/deportivo-alaves/2885',
    'https://www.sofascore.com/team/football/mallorca/2826',
    'https://www.sofascore.com/team/football/rayo-vallecano/2818',
    'https://www.sofascore.com/team/football/sevilla/2833',
    'https://www.sofascore.com/team/football/celta-vigo/2821',
    'https://www.sofascore.com/team/football/cadiz/4488',
    'https://www.sofascore.com/team/football/granada/33779',
    'https://www.sofascore.com/team/football/almeria/2858',
    'https://www.sofascore.com/team/football/bayer-04-leverkusen/2681',
    'https://www.sofascore.com/team/football/fc-bayern-munchen/2672',
    'https://www.sofascore.com/team/football/vfb-stuttgart/2677',
    'https://www.sofascore.com/team/football/borussia-dortmund/2673',
    'https://www.sofascore.com/team/football/rb-leipzig/36360',
    'https://www.sofascore.com/team/football/eintracht-frankfurt/2674',
    'https://www.sofascore.com/team/football/fc-augsburg/2600',
    'https://www.sofascore.com/team/football/tsg-hoffenheim/2569',
    'https://www.sofascore.com/team/football/sc-freiburg/2538',
    'https://www.sofascore.com/team/football/sv-werder-bremen/2534',
    'https://www.sofascore.com/team/football/1-fc-heidenheim/5885',
    'https://www.sofascore.com/team/football/borussia-mgladbach/2527',
    'https://www.sofascore.com/team/football/1-fc-union-berlin/2547',
    'https://www.sofascore.com/team/football/vfl-wolfsburg/2524',
    'https://www.sofascore.com/team/football/vfl-bochum-1848/2542',
    'https://www.sofascore.com/team/football/1-fsv-mainz-05/2556',
    'https://www.sofascore.com/team/football/1-fc-koln/2671',
    'https://www.sofascore.com/team/football/darmstadt-98/2576',
    'https://www.sofascore.com/team/football/inter/2697',
    'https://www.sofascore.com/team/football/milan/2692',
    'https://www.sofascore.com/team/football/juventus/2687',
    'https://www.sofascore.com/team/football/bologna/2685',
    'https://www.sofascore.com/team/football/roma/2702',
    'https://www.sofascore.com/team/football/atalanta/2686',
    'https://www.sofascore.com/team/football/napoli/2714',
    'https://www.sofascore.com/team/football/fiorentina/2693',
    'https://www.sofascore.com/team/football/lazio/2699',
    'https://www.sofascore.com/team/football/monza/2729',
    'https://www.sofascore.com/team/football/torino/2696',
    'https://www.sofascore.com/team/football/genoa/2713',
    'https://www.sofascore.com/team/football/lecce/2689',
    'https://www.sofascore.com/team/football/udinese/2695',
    'https://www.sofascore.com/team/football/hellas-verona/2701',
    'https://www.sofascore.com/team/football/cagliari/2719',
    'https://www.sofascore.com/team/football/empoli/2705',
    'https://www.sofascore.com/team/football/frosinone/2801',
    'https://www.sofascore.com/team/football/sassuolo/2793',
    'https://www.sofascore.com/team/football/salernitana/2710',
    'https://www.sofascore.com/team/football/paris-saint-germain/1644',
    'https://www.sofascore.com/team/football/stade-brestois/1715',
    'https://www.sofascore.com/team/football/as-monaco/1653',
    'https://www.sofascore.com/team/football/lille/1643',
    'https://www.sofascore.com/team/football/nice/1661',
    'https://www.sofascore.com/team/football/lens/1648',
    'https://www.sofascore.com/team/football/olympique-de-marseille/1641',
    'https://www.sofascore.com/team/football/stade-rennais/1658',
    'https://www.sofascore.com/team/football/stade-de-reims/1682',
    'https://www.sofascore.com/team/football/olympique-lyonnais/1649',
    'https://www.sofascore.com/team/football/toulouse/1681',
    'https://www.sofascore.com/team/football/rc-strasbourg/1659',
    'https://www.sofascore.com/team/football/le-havre/1662',
    'https://www.sofascore.com/team/football/montpellier/1642',
    'https://www.sofascore.com/team/football/lorient/1656',
    'https://www.sofascore.com/team/football/nantes/1647',
    'https://www.sofascore.com/team/football/metz/1651',
    'https://www.sofascore.com/team/football/clermont-foot/1680',
        # Add more team URLs here
]

# Collect player URLs for each team
all_player_urls = set()  # Use a set to avoid duplicates across teams
for team_url in team_urls:
    player_urls = find_player_links(team_url)
    all_player_urls.update(player_urls)  # Add player URLs from this team to the overall set

# Convert back to list if needed and print
all_player_urls = list(all_player_urls)
for url in all_player_urls:
    print(url)

https://www.sofascore.com/player/becir-omeragic/940918
https://www.sofascore.com/player/zeki-amdouni/990550
https://www.sofascore.com/player/laurent-abergel/296434
https://www.sofascore.com/player/cedric-bakambu/115665
https://www.sofascore.com/player/nico-williams/1085400
https://www.sofascore.com/player/eder-militao/822519
https://www.sofascore.com/player/dia-boulaye/960080
https://www.sofascore.com/player/nikola-vlasic/357596
https://www.sofascore.com/player/quentin-boisgard/827386
https://www.sofascore.com/player/bentancur-rodrigo/791190
https://www.sofascore.com/player/carney-chukwuemeka/994556
https://www.sofascore.com/player/angelo/1086455
https://www.sofascore.com/player/tyler-onyango/1004342
https://www.sofascore.com/player/lucas-ocampos/155702
https://www.sofascore.com/player/wojciech-szczesny/50490
https://www.sofascore.com/player/pierre-kalulu/965330
https://www.sofascore.com/player/julien-le-cardinal/1129751
https://www.sofascore.com/player/maxim-leitsch/842869
https://www

In [5]:
# Construct the path to chromedriver
current_dir = os.getcwd()  # Get the current working directory
chrome_driver_path = os.path.join(current_dir, "chromedriver-win64", "chromedriver.exe")

In [8]:
# Setup Selenium WebDriver
service = Service(executable_path=os.path.join(os.getcwd(), "chromedriver-win64", "chromedriver.exe"))
driver = webdriver.Chrome(service=service)
driver.fullscreen_window()  # Make the browser window fullscreen

def save_heatmap_image(player_url):
    player_name = player_url.split('/')[-2].replace('-', '_')  # Move this outside the try block
    try:
        driver.get(player_url)
        time.sleep(5)  # Adjust time as needed
        
        # Find the canvas and save its image content
        canvas = driver.find_element(By.ID, 'player-page-heatmap').find_element(By.TAG_NAME, 'canvas')
        canvas_base64 = driver.execute_script("return arguments[0].toDataURL('image/png').substring(21);", canvas)
        canvas_png = base64.b64decode(canvas_base64)
        
        with open(f'{player_name}.png', 'wb') as file:
            file.write(canvas_png)
            
    except NoSuchElementException:
        print(f'Heatmap not found for {player_name}. Skipping...')

# Use the all_player_urls list to download heatmap images
for url in all_player_urls:
    save_heatmap_image(url)

driver.quit()

Heatmap not found for tyler_onyango. Skipping...
Heatmap not found for wharton_adam. Skipping...
Heatmap not found for domingos_duarte. Skipping...
Heatmap not found for denis_suarez. Skipping...
Heatmap not found for franz_stolz. Skipping...
Heatmap not found for gabriel_martinelli. Skipping...
Heatmap not found for meiko_waschenbach. Skipping...
Heatmap not found for mergim_vojvoda. Skipping...
Heatmap not found for bor_zunec. Skipping...
Heatmap not found for razvan_marin. Skipping...
Heatmap not found for silvan_hefti. Skipping...
Heatmap not found for holman_fletcher. Skipping...
Heatmap not found for julian_brandt. Skipping...
Heatmap not found for luke_knapp. Skipping...
Heatmap not found for wesley_okoduwa. Skipping...
Heatmap not found for matteo_brezzo. Skipping...
Heatmap not found for teddy_boulhendi. Skipping...
Heatmap not found for mehmet_avlayici. Skipping...
Heatmap not found for luca_lipani. Skipping...
Heatmap not found for kaua_dos_santos. Skipping...
Heatmap not fo